In [1]:
# !pip install webdriver_manager

In [191]:
#크롤링시 필요한 라이브러리 불러오기
import pandas as pd

from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm


#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [203]:
search_urls = []

st = 20190101000000
month = 100000000

while st < 20229000000000:
    for m in range(11):
        end = st + month
        for p in range(80):
            search_urls.append('https://search.daum.net/search?DA=STC&ed=' + str(end) + '&https_on=on&nil_suggest=btn&p=' + str(p+1) + '&period=u&q=카카오페이&sd=' + str(st) + '&w=news')
        st = end
    end = st + 30000000
    for p in range(80):
            search_urls.append('https://search.daum.net/search?DA=STC&ed=' + str(end) + '&https_on=on&nil_suggest=btn&p=' + str(p+1) + '&period=u&q=카카오페이&sd=' + str(st) + '&w=news') #%20ESG
    st = st + 8900000000



In [204]:
search_urls[-10:]

['https://search.daum.net/search?DA=STC&ed=20221231000000&https_on=on&nil_suggest=btn&p=71&period=u&q=카카오페이&sd=20221201000000&w=news',
 'https://search.daum.net/search?DA=STC&ed=20221231000000&https_on=on&nil_suggest=btn&p=72&period=u&q=카카오페이&sd=20221201000000&w=news',
 'https://search.daum.net/search?DA=STC&ed=20221231000000&https_on=on&nil_suggest=btn&p=73&period=u&q=카카오페이&sd=20221201000000&w=news',
 'https://search.daum.net/search?DA=STC&ed=20221231000000&https_on=on&nil_suggest=btn&p=74&period=u&q=카카오페이&sd=20221201000000&w=news',
 'https://search.daum.net/search?DA=STC&ed=20221231000000&https_on=on&nil_suggest=btn&p=75&period=u&q=카카오페이&sd=20221201000000&w=news',
 'https://search.daum.net/search?DA=STC&ed=20221231000000&https_on=on&nil_suggest=btn&p=76&period=u&q=카카오페이&sd=20221201000000&w=news',
 'https://search.daum.net/search?DA=STC&ed=20221231000000&https_on=on&nil_suggest=btn&p=77&period=u&q=카카오페이&sd=20221201000000&w=news',
 'https://search.daum.net/search?DA=STC&ed=202212310000

In [209]:
from pyvirtualdisplay import Display

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(3)


# selenium으로 검색 페이지 불러오기 #

daum_news=[]

for i in range(int(len(search_urls) / 80)):
    lst = search_urls[i*80 : (i+1)*80]

    for url in tqdm(lst):
        driver.switch_to.window(driver.window_handles[0])
        driver.get(url)
        driver.implicitly_wait(0.5) #대기시간 변경 가능

        check = driver.find_elements(By.CSS_SELECTOR, '#newsColl > div.cont_divider > ul > li:nth-child(10)')
        if len(check) == 1:
            
            for j in range(1, 11, 1):
                driver.switch_to.window(driver.window_handles[0])
                path = '#newsColl > div.cont_divider > ul > li:nth-child(' + str(j) + ') > div.wrap_cont > span.cont_info > a:nth-child(3)'
                a = driver.find_elements(By.CSS_SELECTOR, path)
                if len(a) == 1:
                    time.sleep(1)
                    a[0].click()
                    driver.implicitly_wait(0.5)
                    driver.switch_to.window(driver.window_handles[-1])
                    driver.implicitly_wait(0.5)
                    temp_url = driver.current_url
                    daum_news.append(temp_url)
                    driver.implicitly_wait(0.5)
                    driver.close()
                else:
                    pass
        else:
            print(len(check), url)
            break

<ipython-input-209-fc2638a633d0>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [12:49<00:00,  9.61s/it]


In [210]:
url_df = pd.DataFrame(daum_news)

In [281]:
daum_urls = pd.DataFrame(url_df.iloc[:,1]).drop_duplicates()
daum_urls.reset_index(drop = True, inplace = True)

In [283]:
daum_urls.iloc[:,0]

0        https://v.daum.net/v/20180105100954077
1        https://v.daum.net/v/20180105164004067
2        https://v.daum.net/v/20180105101958491
3        https://v.daum.net/v/20180105173046721
4        https://v.daum.net/v/20180105084208085
                          ...                  
18382    https://v.daum.net/v/20221230105630139
18383    https://v.daum.net/v/20221230071650319
18384    https://v.daum.net/v/20221230052321943
18385    https://v.daum.net/v/20221228120203288
18386    https://v.daum.net/v/20221230091108475
Name: 0, Length: 18387, dtype: object

In [247]:
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

titles = []
contents=[]
for i in tqdm(daum_urls['0']):
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    #print(html)
    
    #뉴스 제목 가져오기
    title = html.select("#mArticle > div.head_view > h3")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='',string=title)
    title = title.replace('[', "")
    title = title.replace(']' , "")
    title = title.replace('"', '')
    #print(title)
    titles.append(title)

    #뉴스 본문 가져오기

    content = html.select("#mArticle > div.news_view.fs_type1")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))
    
    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='',string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,'')
    content = content.replace('\n', '')
    
    #print(content)
    contents.append(content)

100%|████████████████████████████████████████████████████████████████████████████| 18387/18387 [53:40<00:00,  5.71it/s]


In [291]:
lst = []

for u, t, b  in zip(daum_urls['0'], titles, contents):
    lst.append((u,t,b))

In [293]:
news_data = pd.DataFrame(lst)

In [294]:
news_data

,0,1,2
0,https://v.daum.net/v/20180105100954077,카카오페이 오프라인 확장..'카카오페이 카드' 10일 나온다,[카카오페이 카드 2종 온라인 간판결제 '카카오페이'가 '카카오페이 카드'를 출시하...
1,https://v.daum.net/v/20180105164004067,카카오페이 카드 출시,[카카오페이(대표 류영준)는 다양한 온·오프라인 혜택을 담은 혼합형 핀테크 상품 '...
2,https://v.daum.net/v/20180105101958491,"카카오페이, 체크카드 출시..오프라인 진출",[ 【서울=뉴시스】카카오페이는 다양한 온·오프라인 혜택을 담은 ...
3,https://v.daum.net/v/20180105173046721,오프라인 결제 '카카오페이 카드' 출시,[카카오페이 카드 2종 온라인 간판결제 '카카오페이'가 '카카...
4,https://v.daum.net/v/20180105084208085,'카카오페이 카드' 출시.. 다양한 혜택 제공,"[ 이용금액 0.3% 자동 적립, 롯데 ATM 무료 출금 가능카카오페이가 다양한 온..."
...,...,...,...
18382,https://v.daum.net/v/20221230105630139,"파리바게뜨, 새해 1월 ‘파바데이’ 프로모션 진행",[ 1만5000원 이상 구매 시 3000원 혜택설 선물 연계 2만원 이상 구매시 5...
18383,https://v.daum.net/v/20221230071650319,"용인시, 청년 전세보증금 반환보증 보증료 최대 30만원 지원",[ 용인청년 전세보증금 반환보증 보증료 지원사업 홍보 포스터 ...
18384,https://v.daum.net/v/20221230052321943,2금융 대출문턱 막히자… 금융당국 유연하게 대응 당부,[ 2금융권을 중심으로 서민들의 자금조달이 어려워지자 금융당국이...
18385,https://v.daum.net/v/20221228120203288,내년부터 '깜깜이' 간편결제 수수료 공시된다,"[ 금감원, 전자금융업자 수수료 공시 가이드라인 시행네이버페이, 카카오페이 10개사..."


In [295]:
news_data.to_csv("./KP_daum_monthlyTop800_body.csv")